<a href="https://colab.research.google.com/github/ShreyaSutar2004/WetLabBuddy/blob/main/WetLabBuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['HUGGINGFACEHUB_ACCESS_TOKEN']= "//"

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `Shreya` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as

In [ ]:
!pip install huggingface

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver

In [ ]:
!pip install langchain-huggingface

In [ ]:
# INDEXING
API_KEY = "//"
BASE_URL = "https://www.protocols.io/view"

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import JSONLoader, PyPDFLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.prompts import PromptTemplate
import requests
import tempfile
import os
import json

In [ ]:
def fetch_protocol(protocol_id):
    url = f"{BASE_URL}/{protocol_id}"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    print(f"Status Code: {response.status_code}")
    print(f"Response Text: {response.text[:500]}")

    if response.status_code == 200 and 'application/json' in response.headers.get('Content-Type', ''):
        return response.json()
    else:
        raise Exception(f"Failed to fetch protocol: {response.status_code} - {response.text}")


In [ ]:
!curl --compressed \
  -H "Authorization: Bearer //" \
  -H "Accept: application/json" \
  https://www.protocols.io/api/v4/protocols/10.17504/protocols.io.baaciaaw/steps -o steps.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3385    0  3385    0     0   4760      0 --:--:-- --:--:-- --:--:--  4767


In [ ]:
with open('steps.json','r') as file:
  steps= json.load(file)

# Extract the 'text' from each 'block' within the 'step' dictionary for each item in 'payload'
step_texts = []
for step_info in steps['payload']:
    step_data = json.loads(step_info['step'])
    for block in step_data['blocks']:
        if 'text' in block:
            step_texts.append(block['text'])

display(step_texts)

['During site reconnaissance (walking between plots), pay attention to: 1) individuals of rare tree species (see attached Species List per Site and step 11 of the Canopy Trees Survey Protocol - Forests of Southern Québec), and 2) trees that meet the Leaf Spectra selection criteria. ',
 ' ',
 '',
 '',
 'Save your record.',
 ' ',
 '',
 'Under Plants → Plant First Occurence, indicate the names of the team members (one or more) who first observed the plant as well as the date of the first observation of the plant.',
 ' ',
 '',
 'Under Plants → Taxonomic Identification, indicate the team members (one or more) the tree was Identified By, enter the Date Identified, and indicate the Identification Protocol. ',
 ' ',
 '',
 'Under Plants → Location, precisely georeference the tree location by clicking Update location with GPS on your field cellphone connected to the Trimble GPS –coordinates will automatically be imported to the Latitude, Longitude, Horizontal Accuracy and Altitude fields. ',
 ' 

In [ ]:
all_texts = []

for item in steps.get('payload', []):
    step_data = item.get('step')
    if step_data:
        try:

            step_json = json.loads(step_data)


            texts = [block['text'].strip() for block in step_json.get('blocks', []) if block.get('text')]
            all_texts.extend(texts)


            for entity in step_json.get('entityMap', {}).values():
                if entity.get('type') == 'notes':
                    note_blocks = entity['data'].get('blocks', [])
                    for nb in note_blocks:
                        if nb.get('text'):
                            all_texts.append(nb['text'].strip())
        except json.JSONDecodeError:
            continue


clean_protocol_text = "\n".join([t for t in all_texts if t])

print(clean_protocol_text)

During site reconnaissance (walking between plots), pay attention to: 1) individuals of rare tree species (see attached Species List per Site and step 11 of the Canopy Trees Survey Protocol - Forests of Southern Québec), and 2) trees that meet the Leaf Spectra selection criteria.
Leaf Spectra Selection Criteria:
Leaves exposed to direct sunlight for at least 6 hrs a day.
Tree height that allows for leaf collection (target leaves located at ____ m at most).
Within 30 mins, ideally 15 mins, from parking spots for the Leaf Spectra van.
If possible, sample individuals of the same species across the park (i.e., in both sectors and across the altitudinal gradient).
Goal: 10 individual trees per species, for the common species; and 5 individual trees per species, for rare species (see attached Species List).
Details to know to assist the Leaf Spectra Crew during leaf collection:
The Leaf Spectra van, once settled, won’t move for the rest of the day.
Once harvested from the tree, the leaves mu

In [ ]:
protocol_steps = clean_protocol_text


In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 200
)
chunks=splitter.create_documents([protocol_steps])

In [ ]:
len(chunks)

9

In [ ]:
for chunk in chunks :
  print(chunk)
  print(" ")

page_content='During site reconnaissance (walking between plots), pay attention to: 1) individuals of rare tree species (see attached Species List per Site and step 11 of the Canopy Trees Survey Protocol - Forests of Southern Québec), and 2) trees that meet the Leaf Spectra selection criteria.
Leaf Spectra Selection Criteria:
Leaves exposed to direct sunlight for at least 6 hrs a day.
Tree height that allows for leaf collection (target leaves located at ____ m at most).'
 
page_content='Leaf Spectra Selection Criteria:
Leaves exposed to direct sunlight for at least 6 hrs a day.
Tree height that allows for leaf collection (target leaves located at ____ m at most).
Within 30 mins, ideally 15 mins, from parking spots for the Leaf Spectra van.
If possible, sample individuals of the same species across the park (i.e., in both sectors and across the altitudinal gradient).'
 
page_content='If possible, sample individuals of the same species across the park (i.e., in both sectors and across th

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 37.0 MB/s eta 0:00:00


In [ ]:
vector_store=FAISS.from_documents(chunks,embeddings)

In [ ]:
vector_store.index_to_docstore_id

{0: '79156924-46ce-4467-8fa0-a0956cbddc13',
 1: '42b7f164-abc6-43ee-b154-6529ad74413e',
 2: '65d9706c-4ca8-4a0e-b2f4-ee424c678f9a',
 3: 'b0a02afa-e51f-4473-bfc5-3b898d724634',
 4: '0ea53e38-29fa-44ba-a4ac-cd1255e5d266',
 5: 'a3cecaf2-31dc-41cc-8cc0-3196f5812bf6',
 6: '0d0d9a67-808f-410d-9213-83452ccbb4a7',
 7: 'a853923f-6382-4ad0-b903-a7bf5640716d',
 8: '4c80c8b4-0a38-4858-a45b-40df112abe88'}

In [ ]:
vector_store.get_by_ids(['3b4a41b7-0738-42d5-86ae-2c196411af94'])

[]

In [ ]:
from re import search
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':4})

In [ ]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x784224b853d0>, search_kwargs={'k': 4})

In [ ]:
result=retriever.invoke('Write a summary on the given experiment.')
result

[Document(id='0ea53e38-29fa-44ba-a4ac-cd1255e5d266', metadata={}, page_content='If leaf collection occurs before aerial imagery, do not collect leaves inside the plots.\nSave your record.\nUnder Plants → Plant First Occurence, indicate the names of the team members (one or more) who first observed the plant as well as the date of the first observation of the plant.\nUnder Plants → Taxonomic Identification, indicate the team members (one or more) the tree was Identified By, enter the Date Identified, and indicate the Identification Protocol.'),
 Document(id='65d9706c-4ca8-4a0e-b2f4-ee424c678f9a', metadata={}, page_content='If possible, sample individuals of the same species across the park (i.e., in both sectors and across the altitudinal gradient).\nGoal: 10 individual trees per species, for the common species; and 5 individual trees per species, for rare species (see attached Species List).\nDetails to know to assist the Leaf Spectra Crew during leaf collection:\nThe Leaf Spectra van,

In [ ]:
llm = HuggingFaceEndpoint(
    model='google/gemma-2-2b-it',
    task='text-generation'
)

In [ ]:
hf_model=ChatHuggingFace(llm=llm)

In [ ]:
prompt= PromptTemplate(
    template="""
    You're a helpful assistant. Please help the user to understand the {context} of experiment.
    If the user asks {question} but reply and if context is insufficient, say I dont know. """,
    input_variables=['context','question']

)

In [ ]:
retrieved_docs = retriever.invoke('Explain the spectrum criterion.')
retrieved_docs

[Document(id='42b7f164-abc6-43ee-b154-6529ad74413e', metadata={}, page_content='Leaf Spectra Selection Criteria:\nLeaves exposed to direct sunlight for at least 6 hrs a day.\nTree height that allows for leaf collection (target leaves located at ____ m at most).\nWithin 30 mins, ideally 15 mins, from parking spots for the Leaf Spectra van.\nIf possible, sample individuals of the same species across the park (i.e., in both sectors and across the altitudinal gradient).'),
 Document(id='79156924-46ce-4467-8fa0-a0956cbddc13', metadata={}, page_content='During site reconnaissance (walking between plots), pay attention to: 1) individuals of rare tree species (see attached Species List per Site and step 11 of the Canopy Trees Survey Protocol - Forests of Southern Québec), and 2) trees that meet the Leaf Spectra selection criteria.\nLeaf Spectra Selection Criteria:\nLeaves exposed to direct sunlight for at least 6 hrs a day.\nTree height that allows for leaf collection (target leaves located at

In [ ]:
final_context = '\n\n '.join(doc.page_content for doc in retrieved_docs)
final_context

'Leaf Spectra Selection Criteria:\nLeaves exposed to direct sunlight for at least 6 hrs a day.\nTree height that allows for leaf collection (target leaves located at ____ m at most).\nWithin 30 mins, ideally 15 mins, from parking spots for the Leaf Spectra van.\nIf possible, sample individuals of the same species across the park (i.e., in both sectors and across the altitudinal gradient).\n\n During site reconnaissance (walking between plots), pay attention to: 1) individuals of rare tree species (see attached Species List per Site and step 11 of the Canopy Trees Survey Protocol - Forests of Southern Québec), and 2) trees that meet the Leaf Spectra selection criteria.\nLeaf Spectra Selection Criteria:\nLeaves exposed to direct sunlight for at least 6 hrs a day.\nTree height that allows for leaf collection (target leaves located at ____ m at most).\n\n Details to know to assist the Leaf Spectra Crew during leaf collection:\nThe Leaf Spectra van, once settled, won’t move for the rest of 

In [ ]:
final_prompt = prompt.invoke({'context':final_context,'question':'Explain the spectrum criterion.'})
final_prompt

StringPromptValue(text="\n    You're a helpful assistant. Please help the user to understand the Leaf Spectra Selection Criteria:\nLeaves exposed to direct sunlight for at least 6 hrs a day.\nTree height that allows for leaf collection (target leaves located at ____ m at most).\nWithin 30 mins, ideally 15 mins, from parking spots for the Leaf Spectra van.\nIf possible, sample individuals of the same species across the park (i.e., in both sectors and across the altitudinal gradient).\n\n During site reconnaissance (walking between plots), pay attention to: 1) individuals of rare tree species (see attached Species List per Site and step 11 of the Canopy Trees Survey Protocol - Forests of Southern Québec), and 2) trees that meet the Leaf Spectra selection criteria.\nLeaf Spectra Selection Criteria:\nLeaves exposed to direct sunlight for at least 6 hrs a day.\nTree height that allows for leaf collection (target leaves located at ____ m at most).\n\n Details to know to assist the Leaf Spect

In [ ]:
results= hf_model.invoke(final_prompt)
print(results)

content='Okay, I can help explain this. Here\'s a breakdown of the Leaf Spectra Selection Criteria and how to work with them:\n\n**Core Goal: Specific Leaf Samples for Analysis**\n\nThe main idea behind this selection process is to get the highest quality leaf samples for spectral analysis. The goal is to collect leaves that:\n\n* **Get the RIGHT amount of Sun:**  Leaves need direct sunlight for at least 6 hours a day for the "Sunlight Assessments" and the Spectral Analysis.\n* **Are at the Correct Height:** You need leaves from trees within reach, ensuring the truck can safely take samples and that the team can easily work with the leaves without too much physical discomfort.\n* **Are Closest to the Center:** Short and expected travel to the parked Leaf Spectra Van is necessary, for efficiency.\n* **Rely on Better Sample Hopping**: Maintaining sample variety is necessary, so teams should not conserve collection for designated spots, they collect!\n* **Provide Greater Variety:**  Consi

To Build a Chain and Automate


In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableSequence, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def clean_context (retrieved_docs):
  final_context = '\n\n '.join(doc.page_content for doc in retrieved_docs)
  return final_context

In [ ]:
parallel_chain= RunnableParallel(
    {
        'context': retriever | RunnableLambda(clean_context),
        'question': RunnablePassthrough()
    }
)

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = RunnableSequence(parallel_chain | prompt | hf_model | parser)

In [ ]:
main_chain.invoke('Explain the experiment in pointers')

'## Understanding the Leaf Spectra Project: An Assistant\'s Guide \n\nHere\'s a breakdown of the Leaf Spectra project, aimed at helping you succeed in this important ecological study:\n\n**Overall Goal:**\n\nThis project involves measuring chlorophyll content within trees across different sites and altitudes.  The baseline data from these trees will help understand variations in forest health. \n\n**Key Requirements:**\n\n1. **Specimen Collection:** You\'ll collect leaf samples from individual trees. The target is to obtain 10 leaves per common tree species, 5 for rare forest species.\n2. **Species Presence and Altitudinal Analysis:** You\'re required to identify rare species, locate individuals of these species, and assess trees within the chosen site (altitudinal gradients)\n3. **Leaf Spectra Van**\n   - The van serves as the collection and processing hub for your data.\n   - Once in a particular spot, it will not move for the rest of the day, ensuring the collection process is undis